# LangChain: Memory

Language Models are inherently stateless, and they do not remember previous conversations, because they generate next tokens based on the given existing tokens!

That's why we have memory, to keep track of the conversation and feed it to the LLM every time we make a new prompt to it!

## Outline
* ConversationBufferMemory
* ConversationBufferWindowMemory
* ConversationTokenBufferMemory
* ConversationSummaryMemory

## ConversationBufferMemory

In [1]:
# Importing for ENV variables
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

import warnings
warnings.filterwarnings('ignore')

Note: LLM's do not always produce the same results. When executing the code in your notebook, you may get slightly different answers that those in the video.

In [2]:
# account for deprecation of LLM model
import datetime
# Get the current date
current_date = datetime.datetime.now().date()

# Define the date after which the model should be set to "gpt-3.5-turbo"
target_date = datetime.date(2024, 6, 12)

# Set the model variable based on the current date
if current_date > target_date:
    llm_model = "gpt-3.5-turbo" # This in my case
else:
    llm_model = "gpt-3.5-turbo-0301"

In [3]:
# As this is an old course, Langchain synxtax is little different!
# Importing langchain components.
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [4]:
# Creating chat object, Buffer Memory and
# Converstation chain using them!
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True # Let's know how messages are stored!
)

In [5]:
# Starting the conversation!
conversation.predict(input="Hi, i'm Dipesh and i'm very interested in Tech, Space & philosophy!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, i'm Dipesh and i'm very interested in Tech, Space & philosophy!
AI:

> Finished chain.


"Hello Dipesh! It's great to meet someone with such diverse interests. Tech, space, and philosophy are all fascinating topics. What specifically about these subjects interests you the most?"

In [6]:
conversation.predict(input="There uncertainity, yet power to create, awake & destroy")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, i'm Dipesh and i'm very interested in Tech, Space & philosophy!
AI: Hello Dipesh! It's great to meet someone with such diverse interests. Tech, space, and philosophy are all fascinating topics. What specifically about these subjects interests you the most?
Human: There uncertainity, yet power to create, awake & destroy
AI:

> Finished chain.


"That's a very profound observation, Dipesh. The balance between uncertainty and the power to create, awaken, and destroy is indeed a complex and intriguing aspect of life. In philosophy, this concept can be explored through existentialism or even through the study of ethics and morality. In tech, this balance can be seen in the development of artificial intelligence or in the ethical considerations of new technologies. And in space, the unknown and the potential for discovery and exploration highlight this delicate balance as well. It's truly fascinating how these themes intersect across different disciplines."

In [7]:
# Of course it'll remember me now!!
conversation.predict(input="Do you remember? What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, i'm Dipesh and i'm very interested in Tech, Space & philosophy!
AI: Hello Dipesh! It's great to meet someone with such diverse interests. Tech, space, and philosophy are all fascinating topics. What specifically about these subjects interests you the most?
Human: There uncertainity, yet power to create, awake & destroy
AI: That's a very profound observation, Dipesh. The balance between uncertainty and the power to create, awaken, and destroy is indeed a complex and intriguing aspect of life. In philosophy, this concept can be explored through existentialism or even through the study of ethics and morality. In tech, this balance can be seen in the development

'Yes, your name is Dipesh.'

In [8]:
# Show the conversation!
print(memory.buffer)

Human: Hi, i'm Dipesh and i'm very interested in Tech, Space & philosophy!
AI: Hello Dipesh! It's great to meet someone with such diverse interests. Tech, space, and philosophy are all fascinating topics. What specifically about these subjects interests you the most?
Human: There uncertainity, yet power to create, awake & destroy
AI: That's a very profound observation, Dipesh. The balance between uncertainty and the power to create, awaken, and destroy is indeed a complex and intriguing aspect of life. In philosophy, this concept can be explored through existentialism or even through the study of ethics and morality. In tech, this balance can be seen in the development of artificial intelligence or in the ethical considerations of new technologies. And in space, the unknown and the potential for discovery and exploration highlight this delicate balance as well. It's truly fascinating how these themes intersect across different disciplines.
Human: Do you remember? What is my name?
AI: Y

In [9]:
# Load history as a dictionary!
memory.load_memory_variables({})

{'history': "Human: Hi, i'm Dipesh and i'm very interested in Tech, Space & philosophy!\nAI: Hello Dipesh! It's great to meet someone with such diverse interests. Tech, space, and philosophy are all fascinating topics. What specifically about these subjects interests you the most?\nHuman: There uncertainity, yet power to create, awake & destroy\nAI: That's a very profound observation, Dipesh. The balance between uncertainty and the power to create, awaken, and destroy is indeed a complex and intriguing aspect of life. In philosophy, this concept can be explored through existentialism or even through the study of ethics and morality. In tech, this balance can be seen in the development of artificial intelligence or in the ethical considerations of new technologies. And in space, the unknown and the potential for discovery and exploration highlight this delicate balance as well. It's truly fascinating how these themes intersect across different disciplines.\nHuman: Do you remember? What 

In [10]:
# The conversation is stored using this conversation buffer.
# Let's see this
memory = ConversationBufferMemory()

In [11]:
# We can add context like that!
memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})

In [12]:
print(memory.buffer)

Human: Hi
AI: What's up


In [15]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up"}

In [16]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})

In [17]:
memory.load_memory_variables({})

{'history': "Human: Hi\nAI: What's up\nHuman: Not much, just hanging\nAI: Cool"}

### Problems with classical memory!

As we see, LLMs are stateless! Chat bots appear to have memory by providing **entire conversation** as **context** to the next API call to LLM.

But this memory storage can be huge as the conversation grows, and prompting the LLM with this huge context (tokens) every time can cost us a lot!

Hence, Langchain provides convenient Memory types to store the conversations efficiently, let's explore them!!

## ConversationBufferWindowMemory

`ConversationBufferWindowMemory` is a type of memory, that cuts the history of conversation and remembers only `k` recent conversation exchanges ***(Human-AI pairs)***, this way it ensures the conversation memory doesn't grow out of the limit!

Let's see this:

In [18]:
from langchain.memory import ConversationBufferWindowMemory

In [19]:
# K = 1, keep only 1 recent exchange!
memory = ConversationBufferWindowMemory(k=1)               

In [20]:
# Let's manually store 2 converstation exchanges!
# It will drop the earlier one...

memory.save_context({"input": "Hi"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})


In [21]:
memory.load_memory_variables({})

{'history': 'Human: Not much, just hanging\nAI: Cool'}

In [22]:
# Create a Conversation Chain again!
llm = ChatOpenAI(temperature=0.0, model=llm_model)
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True # See how it constantly drops earlier chats!
)

In [23]:
# It won't remember this too long 😥
conversation.predict(input="Hi, I'm Dipesh! I love space!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, I'm Dipesh! I love space!
AI:

> Finished chain.


"Hello Dipesh! I'm glad to hear that you love space. There are so many fascinating things to learn about the universe. Did you know that the closest star to Earth is Proxima Centauri, which is part of the Alpha Centauri star system? It's located about 4.24 light-years away from us. Isn't that amazing?"

In [24]:
conversation.predict(input="Yeah! That's really breath-taking!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, I'm Dipesh! I love space!
AI: Hello Dipesh! I'm glad to hear that you love space. There are so many fascinating things to learn about the universe. Did you know that the closest star to Earth is Proxima Centauri, which is part of the Alpha Centauri star system? It's located about 4.24 light-years away from us. Isn't that amazing?
Human: Yeah! That's really breath-taking!
AI:

> Finished chain.


"I'm glad you think so! There are also many interesting exoplanets that have been discovered in recent years. For example, the exoplanet Kepler-186f is located about 500 light-years away from Earth and is considered to be in the habitable zone of its star. It's exciting to think about the possibility of life beyond our solar system, don't you think?"

In [25]:
conversation.predict(input="I believe so!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Yeah! That's really breath-taking!
AI: I'm glad you think so! There are also many interesting exoplanets that have been discovered in recent years. For example, the exoplanet Kepler-186f is located about 500 light-years away from Earth and is considered to be in the habitable zone of its star. It's exciting to think about the possibility of life beyond our solar system, don't you think?
Human: I believe so!
AI:

> Finished chain.


"There are also other fascinating exoplanets like TRAPPIST-1e, which is one of seven Earth-sized planets orbiting the ultra-cool dwarf star TRAPPIST-1. TRAPPIST-1e is located in the habitable zone of its star and could potentially have liquid water on its surface. The search for exoplanets and the study of their potential habitability is a rapidly growing field in astronomy. It's truly amazing to think about the diversity of worlds that exist beyond our own solar system."

In [27]:
# It doesn't have the context now!
# Fascinating to know how our brains are so powerfull!!
conversation.predict(input="Who am i?!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: I believe so!
AI: There are also other fascinating exoplanets like TRAPPIST-1e, which is one of seven Earth-sized planets orbiting the ultra-cool dwarf star TRAPPIST-1. TRAPPIST-1e is located in the habitable zone of its star and could potentially have liquid water on its surface. The search for exoplanets and the study of their potential habitability is a rapidly growing field in astronomy. It's truly amazing to think about the diversity of worlds that exist beyond our own solar system.
Human: Who am i?!
AI:

> Finished chain.


"I'm sorry, I do not have the ability to determine your identity."

## ConversationTokenBufferMemory

A type of memory that limits the **number of tokens** stored in the conversation! 

This can directly relate with the **cost calculations** of an **LLM via tokens**

In [28]:
# Tokenizer for OpenAI models.
#!pip install tiktoken

In [30]:
# Imports
from langchain.memory import ConversationTokenBufferMemory
from langchain.llms import OpenAI
llm = ChatOpenAI(temperature=0.0, model=llm_model)

In [31]:
# Creating TokenBufferMemory with 50 max_token_limit
# This needs an LLM, to be able to calculate tokens accordingly!
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)

# Saving the context
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

In [32]:
memory.load_memory_variables({}) # See history!

{'history': 'AI: Amazing!\nHuman: Backpropagation is what?\nAI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

We can see, it has **cut down** the first question!
Let's increase the token limit!


To our surprise, it doesn't keeps conversation cut-down in mid!
It keeps the Conversation, correctly understandable!

In [43]:
# Creating TokenBufferMemory with 100 max_token_limit
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=28)

# Saving the context
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})

memory.load_memory_variables({})

{'history': 'AI: Beautiful!\nHuman: Chatbots are what?\nAI: Charming!'}

## ConversationSummaryMemory

A type of memory that also keeps the conversation up to a given **max token limit** but **without cutting down** the conversation exchanges.

Instead, what it does is, when the conversation **overflows** the max tokens, it automatically reduces the conversation to the given tokens by generating a **Summary** of the entire conversation using the LLM itself!

In [45]:
from langchain.memory import ConversationSummaryBufferMemory

In [46]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

# Add the conversation history
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello"}, {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [47]:
memory.load_memory_variables({}) # It generated the summary!

{'history': 'System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool.'}

In [48]:
# Let's test this using a Conversation Chain!
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [49]:
# Starting the conversation.
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool.
Human: What would be a good demo to show?
AI:

> Finished chain.


'For the meeting with the product team, a demo showcasing the latest features and updates on the LangChain project would be ideal. This could include a live demonstration of how LangChain streamlines language translation processes, improves accuracy, and increases efficiency. Additionally, highlighting any recent success stories or case studies would help illustrate the impact of LangChain in real-world scenarios.'

In [51]:
# Again let's ask something!
conversation.predict(input="That's great, what should i focus more on?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: The human and AI exchange greetings and discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool.
Human: What would be a good demo to show?
AI: For the meeting with the product team, a demo showcasing the latest features and updates on the LangChain project would be ideal. This could include a live demonstration of how LangChain streamlines language translation processes, improves accuracy, and increases efficiency. Additionally, highlighting any recent success stories or c

'For the LangChain project work, focusing on the technical aspects of the platform would be beneficial. This could involve discussing the algorithms and machine learning models used in language translation, the data processing pipeline, and any recent improvements or optimizations made to enhance performance. Additionally, highlighting any upcoming features or enhancements in the pipeline could generate excitement and interest from the team.'

In [56]:
memory.load_memory_variables({})

{'history': "System: The human and AI discuss the schedule for the day, including a meeting with the product team, work on the LangChain project, and a lunch meeting with a customer interested in AI. The AI provides details on each event and emphasizes the power of LangChain as a tool. For the meeting with the product team, a demo showcasing the latest features and updates on the LangChain project would be ideal, including a live demonstration of how LangChain streamlines language translation processes, improves accuracy, and increases efficiency. Highlighting recent success stories or case studies would help illustrate the impact of LangChain in real-world scenarios.\nHuman: That's great, what should i focus more on?\nAI: For the LangChain project work, focusing on the technical aspects of the platform would be beneficial. This could involve discussing the algorithms and machine learning models used in language translation, the data processing pipeline, and any recent improvements or 

In [57]:
# Let's show the buffer!
memory.buffer

[HumanMessage(content="That's great, what should i focus more on?", additional_kwargs={}, example=False),
 AIMessage(content='For the LangChain project work, focusing on the technical aspects of the platform would be beneficial. This could involve discussing the algorithms and machine learning models used in language translation, the data processing pipeline, and any recent improvements or optimizations made to enhance performance. Additionally, highlighting any upcoming features or enhancements in the pipeline could generate excitement and interest from the team.', additional_kwargs={}, example=False)]

It's been great!! I read the Langchain documentation also! You should too!

Reminder: Download your notebook to you local computer to save your work.